In [244]:
import requests
from dotenv import dotenv_values
import numpy as np

In [288]:
config = dotenv_values('.env')

In [289]:
def get_cls(data):
    response = requests.post(config['URL_GET_CLS'], json=data)
    list_cls = response.json()
    return list_cls.get('cls_tokens')

In [290]:
cv_skill = np.array(get_cls({"texts" : ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]}))
job_skill = np.array(get_cls({"texts" : ["1", "4", "6", "10"]}))

In [211]:
from sklearn.metrics.pairwise import cosine_similarity

In [212]:
cosine_similarity(cv_skill, job_skill)

array([[1.        , 0.82982182, 0.744114  , 0.64608974],
       [0.86432899, 0.83786612, 0.75525155, 0.56040803],
       [0.84518673, 0.85507918, 0.79233306, 0.55432072],
       [0.82982185, 1.        , 0.80058917, 0.53251584],
       [0.80018296, 0.84208647, 0.81883964, 0.5215729 ],
       [0.74411395, 0.80058926, 1.        , 0.5004912 ],
       [0.75627416, 0.7863666 , 0.81747602, 0.49430584],
       [0.75186295, 0.8024324 , 0.79909361, 0.50433805],
       [0.72491399, 0.75047009, 0.7829723 , 0.51008808],
       [0.6460896 , 0.53251569, 0.50049119, 1.        ]])

In [213]:

# Đầu vào là danh sách văn bản
data = {
    "texts": ["Văn bản đầu tiên", "Văn bản thứ hai"]
}

# Gửi yêu cầu POST tới API
response = requests.post(config['URL_GET_CLS'], json=data)

# Kiểm tra phản hồi
if response.status_code == 200:
    print(response.json())  # In ra CLS token từ API
else:
    print(f"Error: {response.status_code}")  

{'cls_tokens': [[0.2528281509876251, -1.3005950450897217, 0.5861884951591492, -0.38575735688209534, -0.6849125027656555, -0.8526968955993652, 0.03574763610959053, 0.8828121423721313, -0.1960819661617279, -0.9262545108795166, 0.9729778170585632, 0.5055427551269531, 0.5456230044364929, 0.6894980072975159, -1.738372564315796, 0.8024075031280518, 1.973597764968872, -0.1397004872560501, -0.893791675567627, 0.49479883909225464, 0.22134937345981598, -0.5511040091514587, -0.30580800771713257, -0.5868285298347473, -0.2081519067287445, 0.13813184201717377, 0.5752017498016357, 0.4669189751148224, 0.6887167692184448, 0.11190441995859146, 0.011367624625563622, 0.1802091747522354, -0.20649950206279755, -0.9023574590682983, 1.1019243001937866, 0.26358869671821594, 0.24649712443351746, -2.0047545433044434, -0.5060524940490723, -0.30712756514549255, -0.13172227144241333, 0.2921257019042969, 1.378036618232727, -0.17930464446544647, 1.6534054279327393, -1.003222107887268, -0.7816187143325806, 1.066795706

In [304]:
from source.models.job import JobDescription
from source.models.candidate import Candidate
class Rule:
     def __init__(self, job = JobDescription, resume = Candidate):
          self.job = job
          self.resume = resume
          self.job_skills = job.skills
          self.resume_skills = resume.skills
     @staticmethod
     def get_cls(data):
          response = requests.post(config['URL_GET_CLS'], json=data)
          list_cls = response.json()
          return list_cls.get('cls_tokens')
     
     def get_similarity(self,resume_skills, job_skills):
          emb_job_skills = np.array(get_cls({"texts" : job_skills}))
          emb_resume_skills = np.array(get_cls({"texts" : resume_skills}))
          print(emb_job_skills.shape)
          print(emb_resume_skills.shape)
          return cosine_similarity(emb_resume_skills, emb_job_skills)
     
     def get_match(self):
          job_skills = [text.lower() for text in self.job_skills]
          resume_skills = [text.lower() for text in self.resume_skills]
          tmp = []

          result = {
               "job_id": self.job.id,
               "resume_id": self.resume.id,
               "matching_score" : 0,
               "list_matching_skill": []
          }

          average_score = 0
          
          for i, resume_skill in enumerate(resume_skills):
               if resume_skill in job_skills:
                    average_score += 1
                    result["list_matching_skill"].append(resume_skill)
                    job_skills.remove(resume_skill)
               else:
                    tmp.append(resume_skill)

          list_similar_skill = self.get_similarity(tmp, job_skills)
          print(tmp)
          print(job_skills)
          print("list_similar_skill", list_similar_skill)

          average_score = 0
          for score in list_similar_skill:
               for j, value in enumerate(score):
                    if value > 0.9:
                         average_score += 1
                         if self.job_skills[j] not in result["list_matching_skill"]:
                             result["list_matching_skill"].append(self.job_skills[j])

          result["matching_score"] = average_score / len(self.job_skills)
          return result

In [313]:
job = {
     "_id" : '066de609-b04a-4b30-b46c-32537c7f1f6e',
     "role": "Software Engineer",
     "experience": 3.5,
     "acceptable_majors": ["Computer Science"],
     "skills": ["Python", "Java", "C++", "MongoDB", "NLP", "React"]
}

resume = {
     "_id" : '066de609-b04a-4b30-b46c-32537c7f1f6e',
     "contact": {
     "name": "Nguyen Van A",
     "phone_number": "0123456789",
     "email": "exampl1@gmail.com",  # Email will be validated
     "linkedin": "https://www.linkedin.com/in/nguyenvana",
     "location": "Thu Duc, Ho Chi Minh"
     },
     "role":{
          "name": "Software Engineer",
          "num_experience": 3.5
     },
     "language": ["English", "Vietnamese"],
     "skills": ["python", "java", "c++", "ReactJS", "Natural Language Processing(NLP)", "MongoDB"],
     "major": ["Computer Science"]
}

In [314]:
job = JobDescription(**job)
resume = Candidate(**resume)

In [315]:
rule = Rule(job, resume)

In [316]:
rule.get_match()

(2, 768)
(2, 768)
['reactjs', 'natural language processing(nlp)']
['nlp', 'react']
list_similar_skill [[0.52274349 0.82906892]
 [0.41322809 0.49757867]]


{'job_id': '066de609-b04a-4b30-b46c-32537c7f1f6e',
 'resume_id': '066de609-b04a-4b30-b46c-32537c7f1f6e',
 'matching_score': 0.0,
 'list_matching_skill': ['python', 'java', 'c++', 'mongodb']}

In [2]:
import os
import re
import json
from langchain_community.document_loaders import RecursiveUrlLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
from dotenv import load_dotenv

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
url_data = 'https://www.topcv.vn/viec-lam/core-ai-engineer-intern/1553229.html'